In [107]:
import sys
import os

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('analisis.py')))
sys.path.append(BASE_DIR)
from analisis import readNetworkData,drawPiePercentage,drawEntropy

ImportError: cannot import name 'readNetworkData' from 'analisis' (/Users/r/Sites/facu/redes2020/Taller1-Wiretapping/analisis.py)

In [215]:
def readNetworkData(path):
    entropy=[]
    frameData=[]
    with open(path) as file:
        print("read file {0}".format(path))
        frameLines= []
        entropiaLine = ''
        for i, line in enumerate(file):
            if(line.find('Entropia')!=-1):
                en = parseEntropia(line)
            elif(line.find('(')!=-1):
                frameLines.append(parseType(line))
            elif(line.find('frames')!=-1):
                frameLines = parseFrame(line,frameLines)
                frameData = frameData + frameLines
                frameLines=[]
                entropiaLine = ''
                entropy.append(en)
    dfEntropy = pd.DataFrame.from_dict(entropy)
    dfEntropy.columns=['value']
    return pd.DataFrame.from_dict(frameData),dfEntropy
            
def parseEntropia(line):
    return float(line.split(':')[2])

#example of line
# ('UNICAST', 2048) : 0.99942 --- Info:  0.00084
def parseType(line):
    itemType,itemSize = line[line.find('(')+1:line.find(')')].split(', ')
    subs1 = line[line.find(')')+4:]
    value = float(subs1.split('---')[0])
    info= float(subs1.split('---')[1].split(':')[1])
    return {
        'type':itemType,'size':itemSize,'value':value,'info':info       
    }

def parseFrame(line,frameLines):
    frameNumber = int(line.split(' ')[0])
    for frame in frameLines:
        frame.update({'frame':frameNumber})
    
    return frameLines
def drawPiePercentage(df):
    fig = go.Figure(data=[go.Pie(labels=df['type'], values=df['value'])])
    fig.write_html('pie.html', auto_open=True)

def drawProtocols(df):
    fig = go.Figure(data=[go.Bar(x=df['protocol'], y=df['value'])])
    fig.write_html('protocols.html', auto_open=True)
    
    
def drawEntropy(df):
    fig = go.Figure(data=[go.Scatter(y=df['value'])])
    tickXfont = dict(
        font=dict(
            family="Courier New",
            size=14,
            color="#2a3f5f"),
        text='frame'
    )
    xaxis=dict(
        nticks=40,
        tickfont=dict(size=10,family='Courier New'),
        ticks='outside',
        title=tickXfont,
        exponentformat="power",
        showticklabels=True,
        type="linear",
        visible=True,
        rangemode="normal",
        color="#444",
        showexponent="all",
        separatethousands=False,
        ticklen=5,
        tickwidth=1,
        tickcolor="#444",
        linewidth=1,
        gridcolor="#EBF0F8",
        linecolor='#C5CED9',
        gridwidth=1,
        zerolinecolor="#EBF0F8",
        zerolinewidth=2,
        zeroline=True,
        automargin=True,
        layer="above traces",
        fixedrange=False,
        constrain="range",
        constraintoward="center"
    )
    fig.update_layout(xaxis=xaxis,title='Evolucion entropia', yaxis_title="valor entropia",xaxis_title='frame',plot_bgcolor='#FFF' )
    fig.write_html('entropy.html', auto_open=True)
    

In [194]:
!pwd

/Users/r/Sites/facu/redes2020/Taller1-Wiretapping


In [195]:
frames,entropy = readNetworkData('./2020-05-10_11:29:56.out')

read file ./2020-05-10_11:29:56.out


In [196]:
entropy.head(3)

,value
0,0.0
1,0.0
2,0.0


In [197]:
last =  frames[frames['frame']==frames['frame'].max()].copy()

In [198]:
last

,frame,info,size,type,value
133110,22298,0.00356,2048,'UNICAST',0.99753
133111,22298,9.68974,34525,'UNICAST',0.00121
133112,22298,10.98520,2054,'BROADCAST',0.00049
133113,22298,11.63727,35130,'UNICAST',0.00031
133114,22298,11.63727,35020,'UNICAST',0.00031
133115,22298,12.85966,2048,'BROADCAST',0.00013


In [199]:
pie = last.groupby(['frame','type'])['value'].sum().reset_index()
pie

,frame,type,value
0,22298,'BROADCAST',0.00062
1,22298,'UNICAST',0.99936


In [202]:
drawPiePercentage(pie)

In [149]:
drawEntropy(entropy)

In [209]:
def addProtocolName(c):
    if c['type'].find('UNICAST') and int(c['size']) == 2048:
        return 'IPV4'
    elif c['type'].find('UNICAST') and int(c['size']) == 2054:
        return 'ARP'
    elif c['type'].find('UNICAST') and int(c['size']) == 34525:
        return 'IPV6'
    elif c['type'].find('UNICAST') and int(c['size']) == 35130:
        return 'IEEE 1905'
    elif c['type'].find('UNICAST') and int(c['size']) == 35020:
        return 'LLDP'
fra = last.copy()
fra['protocol']= last.apply(addProtocolName,axis=1)

In [212]:
fra.head(10)

,frame,info,size,type,value,protocol
133110,22298,0.00356,2048,'UNICAST',0.99753,IPV4
133111,22298,9.68974,34525,'UNICAST',0.00121,IPV6
133112,22298,10.98520,2054,'BROADCAST',0.00049,ARP
133113,22298,11.63727,35130,'UNICAST',0.00031,IEEE 1905
133114,22298,11.63727,35020,'UNICAST',0.00031,LLDP
133115,22298,12.85966,2048,'BROADCAST',0.00013,IPV4


In [216]:
drawProtocols(fra)

In [121]:
dgroup2 = frames.groupby(['type','size']).count()

In [123]:
dgroup2[dgroup2['size']]

frame   info  value
type        size                      
'BROADCAST' 2048   22268  22268  22268
            2054   22141  22141  22141
'UNICAST'   2048   22298  22298  22298
            34525  22124  22124  22124
            35020  22142  22142  22142
            35130  22143  22143  22143

In [77]:
drawEntropy(entropy)